In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 차량 파손 여부(정상 vs 파손) 분류 파이프라인
# 요구사항
# 1) YOLO pretrained(탐지)로 "차량이 이미지의 50% 이상"인 샘플만 선별
# 2) 선별된 데이터로 (A) YOLO pretrained 기반 분류모델, (B) ResNet pretrained 기반 분류모델 둘 다 학습
# 3) test에서 accuracy / f1 / confusion matrix 보기 좋게 출력
# 4) test에서 TN, FN 샘플을 각 최대 5개 시각화

# =========================
# 0. 설치/임포트
# =========================
!pip -q install ultralytics

import os, random, shutil, math
from pathlib import Path
from collections import defaultdict

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt

from ultralytics import YOLO

# 재현성(선택)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

# =========================
# 1. 사용자 경로 설정
# =========================
# 아래 4개 폴더는 "이미지 파일들이 바로 들어있는" 폴더라고 가정합니다.
NORMAL_DIRS = [
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/normal/images"),
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/normal(kaggle_dataset)"),
]
DAMAGED_DIRS = [
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/damaged/images"),
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/damaged(kaggle_dataset)"),
]

# 필터링/학습용 임시 데이터셋 생성 위치
WORKDIR = Path("/content/gdrive/MyDrive/01.DS Part/99.Study/01.Vehicle_Damage_Detection/Week2_Study")
FILTERED_RAW = WORKDIR / "filtered_raw"     # 필터링된 원본 복사본
CLS_DATASET  = WORKDIR / "cls_dataset"      # ImageFolder 구조( train/val/test / normal|damaged )
YOLO_RUNS    = WORKDIR / "yolo_runs"

# 데이터 split 비율
TRAIN_RATIO = 0.7
VAL_RATIO   = 0.15
TEST_RATIO  = 0.15

# 차량이 차지해야 하는 최소 비율
MIN_CAR_RATIO = 0.50

# YOLO 탐지 모델(Pretrained)
YOLO_DET_WEIGHTS = "yolov8n.pt"   # 필요 시 yolov8s.pt 등으로 변경

# YOLO 분류 모델(Pretrained)
YOLO_CLS_WEIGHTS = "yolov8n-cls.pt"  # 필요 시 yolov8s-cls.pt 등으로 변경

# 학습 하이퍼파라미터(필요 시 조정)
YOLO_EPOCHS = 20
YOLO_IMGSZ  = 224     # 분류는 보통 224/256
YOLO_BATCH  = 32

RESNET_EPOCHS = 10
RESNET_BATCH  = 32
RESNET_LR     = 1e-3
IMG_SIZE      = 224   # ResNet 입력 크기

# =========================
# 2. 유틸: 이미지 파일 수집
# =========================
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPEG",".JPG",".webp"}

def list_images(dir_path: Path):
    if not dir_path.exists():
        print(f"[WARN] Not found: {dir_path}")
        return []
    return [p for p in dir_path.rglob("*") if p.suffix.lower() in IMG_EXTS]

normal_imgs = []
damaged_imgs = []
for d in NORMAL_DIRS:
    normal_imgs += list_images(d)
for d in DAMAGED_DIRS:
    damaged_imgs += list_images(d)

print("normal:", len(normal_imgs))
print("damaged:", len(damaged_imgs))

assert len(normal_imgs) > 0 and len(damaged_imgs) > 0, "이미지 경로를 확인해 주세요."

# =========================
# 3. 1단계: YOLO 탐지로 '차량 비율 >= 50%'만 선별
# =========================
det_model = YOLO(YOLO_DET_WEIGHTS)

# COCO 기준 차량 관련 클래스(필요 시 확장)
# yolov8 COCO names: car, motorcycle, bus, truck 등
VEHICLE_NAMES = {"car", "bus", "truck"}  # 'train' 같은 건 제외

def get_vehicle_ratio_yolo(image_path: Path):
    """
    YOLO 탐지 결과 중 차량 클래스의 bbox 중 가장 큰 bbox 기준으로
    bbox_area / image_area 반환 (차량이 하나도 없으면 0.0)
    """
    try:
        im = Image.open(image_path).convert("RGB")
    except Exception:
        return 0.0

    w, h = im.size
    img_area = float(w * h)

    # stream=False로 단일 이미지 예측
    res = det_model.predict(source=str(image_path), conf=0.25, iou=0.5, verbose=False)
    r = res[0]

    if r.boxes is None or len(r.boxes) == 0:
        return 0.0

    names = r.names  # {id: name}
    best_area = 0.0

    # boxes: xyxy, cls
    xyxy = r.boxes.xyxy.cpu().numpy()
    cls  = r.boxes.cls.cpu().numpy().astype(int)

    for (x1, y1, x2, y2), c in zip(xyxy, cls):
        name = names.get(int(c), "")
        if name in VEHICLE_NAMES:
            bw = max(0.0, x2 - x1)
            bh = max(0.0, y2 - y1)
            area = bw * bh
            if area > best_area:
                best_area = area

    return best_area / img_area if img_area > 0 else 0.0

def filter_images_by_vehicle_ratio(image_paths, min_ratio=0.5):
    kept = []
    dropped = []
    for i, p in enumerate(image_paths, 1):
        ratio = get_vehicle_ratio_yolo(p)
        if ratio >= min_ratio:
            kept.append((p, ratio))
        else:
            dropped.append((p, ratio))
        if i % 200 == 0:
            print(f"  processed {i}/{len(image_paths)} ... kept={len(kept)}")
    return kept, dropped

# 작업 폴더 준비
if WORKDIR.exists():
    shutil.rmtree(WORKDIR)
FILTERED_RAW.mkdir(parents=True, exist_ok=True)

print("\n[Filter] normal ...")
kept_normal, drop_normal = filter_images_by_vehicle_ratio(normal_imgs, MIN_CAR_RATIO)
print("\n[Filter] damaged ...")
kept_damaged, drop_damaged = filter_images_by_vehicle_ratio(damaged_imgs, MIN_CAR_RATIO)

print("\n==== Filter Summary ====")
print(f"Normal kept:  {len(kept_normal)} / {len(normal_imgs)}")
print(f"Damaged kept: {len(kept_damaged)} / {len(damaged_imgs)}")

assert len(kept_normal) > 0 and len(kept_damaged) > 0, "필터 조건이 너무 빡셉니다(차량 50%). MIN_CAR_RATIO를 낮춰보세요."

# =========================
# 4. ImageFolder 구조(train/val/test + class 폴더)로 복사 + split
# =========================
def split_list(items, train_ratio, val_ratio, test_ratio):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6
    items = items[:]
    random.shuffle(items)
    n = len(items)
    n_train = int(n * train_ratio)
    n_val   = int(n * val_ratio)
    train = items[:n_train]
    val   = items[n_train:n_train+n_val]
    test  = items[n_train+n_val:]
    return train, val, test

def safe_copy(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    # 파일명 충돌 방지
    if dst.exists():
        stem = dst.stem
        suf = dst.suffix
        k = 1
        while True:
            cand = dst.with_name(f"{stem}_{k}{suf}")
            if not cand.exists():
                dst = cand
                break
            k += 1
    shutil.copy2(src, dst)

def build_imagefolder_dataset(kept_normal, kept_damaged):
    # (path, ratio) -> path만
    normal_paths = [p for p, _ in kept_normal]
    damaged_paths = [p for p, _ in kept_damaged]

    n_tr, n_va, n_te = split_list(normal_paths, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)
    d_tr, d_va, d_te = split_list(damaged_paths, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

    for split_name, paths, cls_name in [
        ("train", n_tr, "normal"),
        ("val",   n_va, "normal"),
        ("test",  n_te, "normal"),
        ("train", d_tr, "damaged"),
        ("val",   d_va, "damaged"),
        ("test",  d_te, "damaged"),
    ]:
        out_dir = CLS_DATASET / split_name / cls_name
        out_dir.mkdir(parents=True, exist_ok=True)
        for src in paths:
            safe_copy(src, out_dir / src.name)

    print("\n[Dataset built]")
    for split in ["train", "val", "test"]:
        for cls in ["normal", "damaged"]:
            cnt = len(list((CLS_DATASET / split / cls).glob("*")))
            print(f"  {split}/{cls}: {cnt}")

build_imagefolder_dataset(kept_normal, kept_damaged)

In [ ]:
# 차량 파손 여부(정상 vs 파손) 분류 파이프라인
# 요구사항
# 1) YOLO pretrained(탐지)로 "차량이 이미지의 50% 이상"인 샘플만 선별
# 2) 선별된 데이터로 (A) YOLO pretrained 기반 분류모델, (B) ResNet pretrained 기반 분류모델 둘 다 학습
# 3) test에서 accuracy / f1 / confusion matrix 보기 좋게 출력
# 4) test에서 TN, FN 샘플을 각 최대 5개 시각화

# =========================
# 0. 설치/임포트
# =========================
!pip -q install ultralytics

import os, random, shutil, math
from pathlib import Path
from collections import defaultdict

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt

from ultralytics import YOLO

# 재현성(선택)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

# =========================
# 1. 사용자 경로 설정
# =========================
# 아래 4개 폴더는 "이미지 파일들이 바로 들어있는" 폴더라고 가정합니다.
NORMAL_DIRS = [
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/normal/images"),
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/normal(kaggle_dataset)"),
]
DAMAGED_DIRS = [
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/damaged/images"),
    Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/DATA/(share)2026_ImageDetectionStudy_No_resizing/damaged(kaggle_dataset)"),
]

# 필터링/학습용 임시 데이터셋 생성 위치
WORKDIR = Path("/content/gdrive/MyDrive/01.DS Part/99.Study/01.Vehicle_Damage_Detection/Week2_Study")
FILTERED_RAW = WORKDIR / "filtered_raw"     # 필터링된 원본 복사본
CLS_DATASET  = WORKDIR / "cls_dataset"      # ImageFolder 구조( train/val/test / normal|damaged )
YOLO_RUNS    = WORKDIR / "yolo_runs"

# 데이터 split 비율
TRAIN_RATIO = 0.7
VAL_RATIO   = 0.15
TEST_RATIO  = 0.15

# 차량이 차지해야 하는 최소 비율
MIN_CAR_RATIO = 0.50

# YOLO 탐지 모델(Pretrained)
YOLO_DET_WEIGHTS = "yolov8n.pt"   # 필요 시 yolov8s.pt 등으로 변경

# YOLO 분류 모델(Pretrained)
YOLO_CLS_WEIGHTS = "yolov8n-cls.pt"  # 필요 시 yolov8s-cls.pt 등으로 변경

# 학습 하이퍼파라미터(필요 시 조정)
YOLO_EPOCHS = 20
YOLO_IMGSZ  = 224     # 분류는 보통 224/256
YOLO_BATCH  = 32

RESNET_EPOCHS = 10
RESNET_BATCH  = 32
RESNET_LR     = 1e-3
IMG_SIZE      = 224   # ResNet 입력 크기

# =========================
# 5-A. YOLO 분류모델 학습 (Ultralytics)
# =========================
yolo_cls = YOLO(YOLO_CLS_WEIGHTS)

# Ultralytics classify는 data=에 "train 폴더"를 넣으면 (val 포함) 학습합니다.
# 여기서는 CLS_DATASET/train 사용 + val은 자동으로 CLS_DATASET/val을 찾기도 하지만,
# 가장 안전하게는 data=CLS_DATASET 로 루트 구조를 주는 방식이 편합니다.
# (Ultralytics 버전에 따라 동작 차이가 있을 수 있어, 아래는 호환성 높은 방식)

yolo_exp_dir = YOLO_RUNS / "yolo_cls"
yolo_results = yolo_cls.train(
    data=str(CLS_DATASET),   # 루트에 train/val/test가 있으면 분류에서 인식 가능
    epochs=YOLO_EPOCHS,
    imgsz=YOLO_IMGSZ,
    batch=YOLO_BATCH,
    project=str(YOLO_RUNS),
    name="yolo_cls",
    verbose=True
)

# 가장 좋은 weights 경로(보통 runs/classify/.../weights/best.pt)
yolo_best = yolo_cls.trainer.best
print("YOLO best weights:", yolo_best)

# =========================
# 5-B. ResNet(사전학습) 분류모델 학습 (Torchvision)
# =========================
train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
eval_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

train_ds = datasets.ImageFolder(CLS_DATASET / "train", transform=train_tf)
val_ds   = datasets.ImageFolder(CLS_DATASET / "val",   transform=eval_tf)
test_ds  = datasets.ImageFolder(CLS_DATASET / "test",  transform=eval_tf)

class_to_idx = train_ds.class_to_idx  # {'damaged':0, 'normal':1} 처럼 될 수 있음
idx_to_class = {v:k for k,v in class_to_idx.items()}
print("class_to_idx:", class_to_idx)

train_loader = DataLoader(train_ds, batch_size=RESNET_BATCH, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=RESNET_BATCH, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=RESNET_BATCH, shuffle=False, num_workers=2, pin_memory=True)

resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet.fc = nn.Linear(resnet.fc.in_features, 2)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=RESNET_LR)

def run_epoch(model, loader, train=False):
    model.train(train)
    total_loss, total_correct, total = 0.0, 0, 0
    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        if train:
            optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        if train:
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(1)
        total_correct += (pred == y).sum().item()
        total += x.size(0)
    return total_loss/total, total_correct/total

best_val_acc = -1
best_path = WORKDIR / "resnet18_best.pt"

for epoch in range(1, RESNET_EPOCHS+1):
    tr_loss, tr_acc = run_epoch(resnet, train_loader, train=True)
    va_loss, va_acc = run_epoch(resnet, val_loader,   train=False)
    print(f"[ResNet] epoch {epoch:02d} | train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")
    if va_acc > best_val_acc:
        best_val_acc = va_acc
        torch.save(resnet.state_dict(), best_path)

print("Best val acc:", best_val_acc, "saved:", best_path)

In [ ]:
# =========================
# 6. 평가 지표(accuracy, f1, confusion matrix) 공통 함수
# =========================
def confusion_matrix_binary(y_true, y_pred, positive_label):
    """
    positive_label: 'damaged' 같은 양성 클래스 이름
    """
    # 라벨을 0/1로
    y_true_bin = np.array([1 if t==positive_label else 0 for t in y_true])
    y_pred_bin = np.array([1 if p==positive_label else 0 for p in y_pred])

    TP = int(((y_true_bin==1) & (y_pred_bin==1)).sum())
    TN = int(((y_true_bin==0) & (y_pred_bin==0)).sum())
    FP = int(((y_true_bin==0) & (y_pred_bin==1)).sum())
    FN = int(((y_true_bin==1) & (y_pred_bin==0)).sum())
    cm = np.array([[TN, FP],[FN, TP]])  # [[TN,FP],[FN,TP]]
    return cm, TP, TN, FP, FN

def accuracy_score(y_true, y_pred):
    return float((np.array(y_true) == np.array(y_pred)).mean())

def f1_score_binary(y_true, y_pred, positive_label):
    cm, TP, TN, FP, FN = confusion_matrix_binary(y_true, y_pred, positive_label)
    precision = TP / (TP + FP + 1e-12)
    recall    = TP / (TP + FN + 1e-12)
    f1 = 2 * precision * recall / (precision + recall + 1e-12)
    return float(f1), float(precision), float(recall)

def pretty_print_metrics(title, acc, f1, precision, recall, cm, labels=("normal","damaged")):
    print("\n" + "="*60)
    print(title)
    print("="*60)
    print(f"Accuracy : {acc:.4f}")
    print(f"F1 (pos=damaged) : {f1:.4f}   (Precision {precision:.4f}, Recall {recall:.4f})")
    print("\nConfusion Matrix (rows=true, cols=pred)")
    print(f"          pred {labels[0]:>8}   pred {labels[1]:>8}")
    print(f"true {labels[0]:>8}   {cm[0,0]:8d}      {cm[0,1]:8d}")
    print(f"true {labels[1]:>8}   {cm[1,0]:8d}      {cm[1,1]:8d}")

# =========================
# 7-A. YOLO 분류모델 test 예측 + 지표
# =========================
# YOLO predict는 폴더를 넣으면 각 이미지별 top1 예측을 줍니다.
# test 폴더 내부의 모든 이미지를 모아서 예측
test_img_paths = list((CLS_DATASET/"test").rglob("*"))
test_img_paths = [p for p in test_img_paths if p.suffix.lower() in IMG_EXTS]

def get_true_label_from_path(p: Path):
    # .../test/normal/xxx.jpg 또는 .../test/damaged/xxx.jpg
    parts = p.parts
    # 마지막에서 두번째가 class 폴더라고 가정
    return parts[-2]

# YOLO best weights 로딩
yolo_cls_best = YOLO(str(yolo_best))

y_true_yolo = []
y_pred_yolo = []

# 배치 예측(속도 위해)
yolo_pred = yolo_cls_best.predict([str(p) for p in test_img_paths], verbose=False)
for p, r in zip(test_img_paths, yolo_pred):
    true = get_true_label_from_path(p)
    # r.probs.top1 == 클래스 index
    top1 = int(r.probs.top1)
    pred_name = r.names[top1]
    y_true_yolo.append(true)
    y_pred_yolo.append(pred_name)

acc_yolo = accuracy_score(y_true_yolo, y_pred_yolo)
f1_yolo, prec_yolo, rec_yolo = f1_score_binary(y_true_yolo, y_pred_yolo, positive_label="damaged")
cm_yolo, TP, TN, FP, FN = confusion_matrix_binary(y_true_yolo, y_pred_yolo, positive_label="damaged")

pretty_print_metrics("[YOLO-CLS] Test metrics", acc_yolo, f1_yolo, prec_yolo, rec_yolo, cm_yolo, labels=("normal","damaged"))

# =========================
# 7-B. ResNet test 예측 + 지표
# =========================
# best 모델 로딩
resnet.load_state_dict(torch.load(best_path, map_location=DEVICE))
resnet.eval()

# ImageFolder(test)는 샘플 순서와 (이미지경로,라벨) 매핑을 제공
test_samples = test_ds.samples  # [(path, class_idx), ...]

y_true_res = []
y_pred_res = []

with torch.no_grad():
    for i in range(0, len(test_ds), RESNET_BATCH):
        batch = [test_ds[j] for j in range(i, min(i+RESNET_BATCH, len(test_ds)))]
        xs = torch.stack([b[0] for b in batch]).to(DEVICE)
        ys = torch.tensor([b[1] for b in batch]).to(DEVICE)
        logits = resnet(xs)
        preds = logits.argmax(1)

        y_true_res += [idx_to_class[int(t)] for t in ys.cpu().numpy().tolist()]
        y_pred_res += [idx_to_class[int(p)] for p in preds.cpu().numpy().tolist()]

acc_res = accuracy_score(y_true_res, y_pred_res)
f1_res, prec_res, rec_res = f1_score_binary(y_true_res, y_pred_res, positive_label="damaged")
cm_res, TP, TN, FP, FN = confusion_matrix_binary(y_true_res, y_pred_res, positive_label="damaged")

pretty_print_metrics("[ResNet18] Test metrics", acc_res, f1_res, prec_res, rec_res, cm_res, labels=("normal","damaged"))

# =========================
# 8. TN / FN 샘플 시각화 (각 최대 5개)
# - positive = damaged, negative = normal 가정
# - TN: true normal & pred normal (정상 맞춘 예)
# - FN: true damaged & pred normal (파손을 놓친 예)
# =========================
def (test_paths, y_true, y_pred, sample_type, max_n=5, pos="damaged", neg="normal"):
    picked = []
    for p, t, pr in zip(test_paths, y_true, y_pred):
        if sample_type == "TN":
            if (t==neg) and (pr==neg):
                picked.append((p, t, pr))
        elif sample_type == "FN":
            if (t==pos) and (pr==neg):
                picked.append((p, t, pr))
        if len(picked) >= max_n:
            break
    return picked

def show_samples(samples, title):
    if len(samples) == 0:
        print(f"\n[{title}] 샘플이 없습니다.")
        return
    n = len(samples)
    plt.figure(figsize=(4*n, 4))
    for i, (p, t, pr) in enumerate(samples, 1):
        img = Image.open(p).convert("RGB")
        plt.subplot(1, n, i)
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"{p.name}\ntrue={t}, pred={pr}")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# YOLO용 TN/FN
tn_yolo = pick_samples_by_type(test_img_paths, y_true_yolo, y_pred_yolo, "TN", max_n=5)
fn_yolo = pick_samples_by_type(test_img_paths, y_true_yolo, y_pred_yolo, "FN", max_n=5)

show_samples(tn_yolo, "[YOLO-CLS] TN samples (max 5)")
show_samples(fn_yolo, "[YOLO-CLS] FN samples (max 5)")

# ResNet용 TN/FN
# ResNet은 test_ds.samples의 path 순서와 y_true_res/y_pred_res가 일치
test_paths_res = [Path(p) for p, _ in test_samples]
tn_res = pick_samples_by_type(test_paths_res, y_true_res, y_pred_res, "TN", max_n=5)
fn_res = pick_samples_by_type(test_paths_res, y_true_res, y_pred_res, "FN", max_n=5)

show_samples(tn_res, "[ResNet18] TN samples (max 5)")
show_samples(fn_res, "[ResNet18] FN samples (max 5)")

# =========================
# 9. 참고: 자주 조정하는 포인트
# =========================
# (1) 필터가 너무 엄격해서 데이터가 적으면:
#     - MIN_CAR_RATIO = 0.4 또는 0.3 으로 낮추기
#     - VEHICLE_NAMES에 'person' 같은 걸 넣지 말고 차량 클래스만 유지
#
# (2) YOLO 탐지 성능이 부족하면:
#     - YOLO_DET_WEIGHTS를 yolov8s.pt / yolov8m.pt로 변경(정확도↑, 속도↓)
#
# (3) 분류 성능이 부족하면:
#     - YOLO_CLS_WEIGHTS를 yolov8s-cls.pt로 변경
#     - YOLO_EPOCHS/RESNET_EPOCHS 증가
#     - 이미지 증강(RandomRotation, ColorJitter 등) 추가
#
# (4) TN은 '틀린 이미지'가 아니라 '정상 맞춘 이미지'입니다.
#     사용자가 원하신 "두 유형"을 그대로 보여드리기 위해 TN/FN을 시각화했습니다.
#     만약 "틀린 것만" 보려면 FP/FN을 시각화하도록 바꾸면 됩니다.


Output hidden; open in https://colab.research.google.com to view.